In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv
import time

In [3]:
def get_html(df_hrefs):
    
    base_url ='https://www.oreillyauto.com'
    # Iterate through the rows and create full URLs
    full_urls = [base_url + x for x in df_hrefs.href.to_list()]

    return full_urls    

In [4]:
def get_data(link):    
    
    headers = {
        'authority': 'www.oreillyauto.com',
        'cache-control': 'max-age=0',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',

        'referer': 'http://localhost:8888/',
        'accept-language': 'es-419,es;q=0.9',
        'cookie': 'selectedStoreId=6361'
    }
    
    response = requests.get(link, headers=headers)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    details = soup.find('div', class_ = 'pdp-price-avail')
    
    try:
        product_name = details.find('h1',class_ = 'page-title condensed js-product-name js-ga-product-name').text
    
    except:
        product_name = 'error'
        pass

    price = details.find('strong',class_ = 'pricing_price js-ga-product-price').text.lstrip('$')


    try:
        availability = details.find('label',class_ = 'avail_title')
        free_pickup_element = availability.find('span', class_='free-pickup')
        pickup_in_store_element = free_pickup_element.find_next_sibling('span').text
        available = 'Pick Up in Store'

    except:
        available = 'no'
        pass
               
    #     Pick Up in Store
    elem = {
        'product_name': product_name, 
        'price_usd': price,
        'available': available,
     #   'reviews': reviews
        }
    return elem

In [ ]:
df_hrefs = pd.read_csv('hrefs.csv')

In [5]:
# 10.000 links to scrape. We take a sample of 1000
urls = get_html(df_hrefs)
urls_test = urls [0:1000]

In [6]:
elements = []
exceptions = []
for i, link in enumerate(urls_test): 
    try:
        elements.append(get_data(link))
        time.sleep(0.5)
    except Exception as e:
        exceptions.append({
        'iter': i,
        'link': link,
        'exception': e 
        })        

In [7]:
df = pd.DataFrame(elements)
df

,product_name,price_usd,available
0,Power Torque Cylinder Head,799.00,no
1,Sierra Marine Ignition Coil,60.99,Pick Up in Store
2,Cardone Secondary Air Injection Pump - Remanuf...,314.99,Pick Up in Store
3,Standard Ignition 1 Terminal Auto Trans Kickdo...,36.99,Pick Up in Store
4,Century Drill Bit,9.99,Pick Up in Store
...,...,...,...
986,Gates Synthetic Rubber Coolant Hose,75.99,Pick Up in Store
987,Gates Mega Crimp Adapter,7.49,Pick Up in Store
988,Import Direct Fuel Pump Module Assembly,235.99,Pick Up in Store
989,Precision Coil Spring Set,101.99,Pick Up in Store


In [8]:
df_exceptions = pd.DataFrame(exceptions)
df_exceptions

,iter,link,exception
0,13,https://www.oreillyauto.com/detail/c/ngk/ngk-r...,'NoneType' object has no attribute 'find'
1,22,https://www.oreillyauto.com/detail/c/power-tor...,'NoneType' object has no attribute 'find'
2,60,https://www.oreillyauto.com/detail/c/masterpro...,'NoneType' object has no attribute 'find'
3,150,https://www.oreillyauto.com/detail/c/wix/wix-f...,'NoneType' object has no attribute 'find'
4,486,https://www.oreillyauto.com/detail/c/acdelco/a...,'NoneType' object has no attribute 'find'
5,652,https://www.oreillyauto.com/detail/c/hold-zit/...,'NoneType' object has no attribute 'find'
6,847,https://www.oreillyauto.com/detail/c/drivealig...,'NoneType' object has no attribute 'find'
7,914,https://www.oreillyauto.com/detail/c/powergrip...,'NoneType' object has no attribute 'find'
8,976,https://www.oreillyauto.com/detail/c/masterpro...,'NoneType' object has no attribute 'find'


In [9]:
df.to_csv('oreilly_data.csv', index=False)
df_exceptions.to_csv('exceptions.csv', index=False)